<a href="https://colab.research.google.com/github/xx529/Others/blob/main/Practice/JAX%20Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import matplotlib.pyplot as plt
import time

In [2]:
digits = datasets.load_digits()
features = digits['data']
target = digits['target'] 

In [3]:
train_x, test_x, train_y, test_y = train_test_split(features, target, test_size=0.33)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1203, 64)
(1203,)
(594, 64)
(594,)


# Pytorch

In [4]:
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset

In [5]:
# 格式转化
pytorch_train_x = torch.tensor(train_x, dtype=torch.float32)
pytorch_train_y = torch.tensor(train_y, dtype=torch.long)

pytorch_test_x = torch.tensor(test_x, dtype=torch.float32)
pytorch_test_y = torch.tensor(test_y, dtype=torch.long)

# 装进数据容器分批
data = TensorDataset(pytorch_train_x, pytorch_train_y)
dataset = DataLoader(data, 64)

In [17]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.relu(self.fc6(x))
        result = self.fc7(x)
        return result
    
    def predict(self, x):
        result = self.softmax(self(x)).sort()[1][:,-1]
        return result

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
optimizer = optim.Adam(model.parameters())
loss_fun = nn.CrossEntropyLoss()

In [19]:
%%time
# 训练
for i in range(25):
    loss_record = 0
    start_time = time.time()
    
    for batch, (x, y) in enumerate(dataset):
        optimizer.zero_grad()

        x, y = x.to(device), y.to(device)
        y_pre = model(x)
        
        loss = loss_fun(y_pre, y)
        loss.backward()
        optimizer.step()
        
        loss_record += loss.item()
    
    duration = time.time() - start_time
    
    predict = model.predict(pytorch_test_x).numpy()
    acc_score = accuracy_score(predict, pytorch_test_y.numpy())
    
    print('Epoch: {}, time: {:.3f}s, Loss: {:.3f}, Acc: {:.2f}%'.format(i+1, duration, loss_record/batch, acc_score*100))

RuntimeError: ignored